## Application Header

In [1]:
# -*- coding: utf-8 -*-

"""
 an interactive program (text-based) for a vending machine that can dispense 
 various products like tea, coffee, snacks, and cold drinks after coins are inserted.

"""

# Imports
import copy
import pandas as pd
from datetime import datetime
import math
from tabulate import tabulate
import matplotlib.pyplot as plt

__author__ = 'Parsa Mousavi SMOU0003'
__copyright__ = 'Copyright 2023, Assessment 2'
__version__ = '1.0.0'

## Reset Files
These functions reset the files back to their default values if needed by the admin

In [2]:
def ResetStock():
    data = {'Item': ['1.Tea', '2.Cappuccino', '3.Latte', '4.Espresso', '5.Coke', '6.Juice', '7.Sugar'],
        'Stock': [20, 50, 15, 20, 20, 15, 50],
        'Cost': [2.5, 3.0, 2.5, 2.0, 2.5, 2.0, 0]}
    
    df = pd.DataFrame(data)
    df.to_csv('Stock.csv', index=False)
    
def ResetTill():
    data = {'Coin': ['$2', '$1', '50c', '20c', '10c'],
        'Quantity': [20, 20, 50, 50, 100],}
    
    df = pd.DataFrame(data)
    df.to_csv('Till.csv', index=False)

### Read Files
These functions are responsible for reading/creating the system files, stock, money till and financial records 

In [3]:
def ReadStock():
    while True:
        try:
            with open('Stock.csv', 'r') as file:
                Stock = pd.read_csv(file)
            return Stock
        except:
            ResetStock()
            continue

def ReadTill():
    while True:
        try:
            with open('Till.csv', 'r') as file:
                Till = pd.read_csv(file)
            return Till
        except:
            ResetTill()
            continue

def ReadRecords():
    try:
        with open("Records.csv", "r") as file:
            Records = pd.read_csv(file)
        return Records
    except:
        df = pd.DataFrame(columns=['Date', 'Time', 'Items', 'Total','Coins'])
        df.to_csv("Records.csv", index=False)
        with open("Records.csv", "r") as file:
            Records = pd.read_csv(file)
            
        return Records
    
    
Stock = ReadStock()
Till = ReadTill()
Records = ReadRecords()

## Sales Diagram
This function when called will generate a line diagram by reading data from the Records file.
The plot will be dates versus sum of sales for that date

In [15]:
def SalesDiagram():
    # Function to format totals from records
    def FormatCurrency(total):
        return float(total.replace('$', ''))
    data = pd.read_csv("Records.csv", parse_dates=['Date'], converters={'Total': FormatCurrency})
    data = data.groupby('Date')['Total'].sum().reset_index()

    plt.plot(data['Date'], data['Total'], marker = 'o')
    plt.xlabel('Date')
    plt.ylabel('Sales $')
    plt.title('Total Sales by Date')
    plt.show()

### Create Menu
This block reads the stock and initiates the menu dictionary.

In [5]:
def CreateMenu():
    while True:
        try:
            menu = {'Item':[], 'Stock':[], 'Cost':[]}
            items = []
            stocks = []
            costs = []

            for index, row in Stock.iterrows():
                items.append(row['Item'])
                stocks.append(row['Stock'])
                costs.append(row['Cost'])

            menu['Item'] = items
            menu['Stock'] = stocks
            menu['Cost'] = costs
    
            return menu
        
        except:
            ResetStock()
            continue

menu = CreateMenu()

## Display Menu
A function that prints the menu and displays out of Stock items it takes a dictionary input

In [7]:
def DisplayMenu(menu):
    while True:
        try:
            print("------ MENU ------")

            for name, price, stock in zip(menu['Item'], menu['Cost'], menu['Stock']):  
                if (name != '7.Sugar' and stock > 0):
                    print(name,"$",price)

            print("------------------")        
            for name, price, stock in zip(menu['Item'], menu['Cost'], menu['Stock']):  
                if (name != "7.Sugar" and stock == 0 ):
                    print(name, "- Unavailable")
            
            break
        except:
            ResetStock()
            continue

### Item Selection
This function handles the items and quantities selected.

In [8]:
def ItemSelection(menu):
    Order = {}
    SelectionActive = True
    while(SelectionActive): 
        try:
            Item = None
            Quantity = 0
            while True:
                selection = input("\nEnter the item number: ")
                for name, price, stock in zip(menu['Item'], menu['Cost'], menu['Stock']):  
                    if (selection in name and stock > 0 and name != "7.Sugar" and selection != ""):
                        Item = name 
                        break

                if(Item == None):
                    print("Please select only from the available items by entering the item number")
                    continue
                else:
                    break

            print(f"You selected {Item}")
            
            while True:
                try:
                    Qty = int(input("Enter the quantity for the item: "))
                    index = menu['Item'].index(Item)
                    
                    if (Qty == 0):
                        print("Quantity cannot be 0!")
                        continue
                    elif (Qty <= menu['Stock'][index]):
                        Quantity = Qty
                        menu['Stock'][index] = menu['Stock'][index] - Quantity
                        break
                    else:
                        print(f"Sorry we only have {menu['Stock'][index]} in stock.")
                        continue

                except:
                    print("Please enter a correct value")
                    continue
        
            Order[Item] = Quantity
          
            while True:
                selection = input("Would you like to select more items? yes/no: ").lower()
                    
                if(selection == 'no'):
                    SelectionActive = False
                    break
                elif(selection == 'yes'):
                    break
                else:
                    print("Error, Please enter yes or no.")
                    continue
                        
        except:
            print("Error")
    
    return Order

## Display Cost
This function reads the items in the cart and displays their cost. returns cost.

In [9]:
def DisplayCost(Order):
    cost = 0 
    print("\nTransaction Overview: ")
    print("------------------------")
    print("Item x Quantity | Amount")
    for key, value in Order.items():
        index = menu['Item'].index(key)
        amount = float(menu['Cost'][index]) * value
        print(f"{key} x {value} | ${amount}")
        cost += amount
    print("________________________")
    print(f"Total | ${cost}")
    return cost

## Options Button
This function serves as a button for the user to use at any step of the transaction. It accepts 2 string inputs, Message is used at each checkpoint the button is called in context to that checkpoint. Message2 is optional and it's primarily for the payment screen.

In [10]:
def Options(Message, Message2=""):
    ButtonActive = True
    while(ButtonActive):
        try:
            answer = input(f"Select one:\n Y.Continue {Message},{Message2} C. Cancel, R. Reset Transaction: ").lower()
            
            if(answer == "y"):
                ButtonActive = False
                return "y"
            elif(answer == "c"):
                ButtonActive = False
                return "c"
            elif(answer == "r"):
                ButtonActive = False
                return "r"
            elif(answer == "m"):
                ButtonActive = False
                return "m"
            else:
                print("Please select one of the options")
            
        except:
            print("Error, Please select one of the options")
            continue

## Coin Slot
The function accepts total order cost as input.
This function accepts and validates validates payments by user and returns any change. It tracks what coins were given for the payment.

In [11]:
def CoinSlot(Total,CoinsPaid):
    AcceptedCoins = ["$2", "$1", "50c", "20c", "10c"]
    print("\nAccepted coins, $2, $1, 50c, 20c, 10c")
    Inserting = True
    Payment = 0
    
    while(Inserting):
         try:
            print("Insert coins: \n")
            for i in range(0,5):
                ValidatingCoin = True
                while(ValidatingCoin):
                    try:
                        Coins = int(input(f"{AcceptedCoins[i]} coin(s): "))
                        if(AcceptedCoins[i] == "$2"):
                            CoinsPaid[i] = Coins
                            Payment += (Coins * 2)
                            break
                        elif(AcceptedCoins[i] == "$1"):
                            CoinsPaid[i] = Coins
                            Payment += Coins
                            break
                        elif(AcceptedCoins[i] == "50c"):
                            CoinsPaid[i] = Coins
                            Payment += (Coins * 0.5)
                            break
                        elif(AcceptedCoins[i] == "20c"):
                            CoinsPaid[i] = Coins
                            Payment += (Coins * 0.2)
                            break
                        elif(AcceptedCoins[i] == "10c"):
                            CoinsPaid[i] = Coins
                            Payment += (Coins * 0.1)
                            ValidatingCoin = False
                            break
                    except:
                        print("Please enter the correct currency.")
                        continue
                
            if(Payment >= Total):
                print("Payment complete, Thank you!")
                ChangeOutcome = CalculateChange(Payment, Total, CoinsPaid)
                if (ChangeOutcome == "Failed"):
                    Inserting = False
                    return "f"
                else:
                    Inserting = False
                    return Payment
            elif(Payment <= Total):
                Remaining = Total - Payment
                print(f"\nInsufficient funds, would you like to pay the remaining amount of ${Remaining:.1f}?")
                Choice = Options("To Pay Remaining Balance", " M. Modify Cart,")
                    
                if(Choice == "c"):
                    Inserting = False
                    print("\nTransaction cancelled, please take your change.")
                    return "c"
                elif(Choice == "y"):
                    continue
                elif(Choice == "r"):
                    Inserting = False
                    return "r"
                elif(Choice == "m"):
                    print("\nPayment cancelled, please take your change.")
                    Inserting = False
                    return "m"
        
         except:
            print("Please enter the correct currency.")
            continue

## Modify Cart
This function accepts the order dictionary as a parameter. It serves allow users to change the quantity of items in their order.

In [10]:
def ModifyCart(Order):
    print("\nCurrent items in your cart:")
    print(Order)
    Modifying = True
    
    while(Modifying):
        try:
            ModifyingItems = True
            while(ModifyingItems):
                Item = input("Enter the number of the item you would like to modify: ")
                try:
                    for key,value in Order.items():
                        if Item in key:
                            ModifyingItems = False
                            break
                        else:
                            print("Please select a valid item.\n")
                            continue
                            
                except:
                    print("Please select a valid item.\n")
                    continue
            
            
            ModifyingQty = True
            while(ModifyingQty):
                try:
                    for key,value in Order.items():
                        if Item in key:
                            Choice = int(input("Enter the new quantity (set to 0 to remove item): "))
                            if (Choice == 0):
                                del Order[key]
                            else:
                                Order[key] = Choice
                            ModifyingQty = False
                            break
                except:
                    print("Please select a valid quantity.\n")
                    continue
                    
            while(Modifying):
                try:
                    Choice = input("Would you like to change another item? yes/no ").lower()
                    if(Choice == "yes"):
                        break
                    elif(Choice == "no"):
                        Modifying = False
                        break
                except:
                    print("Please enter yes or no.\n")
                    continue
        
        except:
            print("Please select a valid item.\n")
            continue

## Sugar Mixer
Provide the options for adding sugar to hot beverages and display time message

In [37]:
def SugarOptions(Order, tempMenu):
    for key, value in Order.items():
        if (key != "5.Coke" and key != "6.Juice"):
            while True:
                try:
                    Choice = input(f"\nWould you like to add sugar to your {key[2:]}(s)? yes/no ").lower()
                    
                    if (Choice == "yes" and tempMenu['Stock'][6] > 0):
                        tempMenu['Stock'][7] -= Order[key]
                        while True:
                            try:
                                Choice = input(f"\nWould you like it to be mixed automatically with the {key[2:]}(s) or" 
                                               "add it yourself? automatic/myself").lower()

                                if (Choice == "automatic"):
                                    print("\nYou have selected automatic mixing!")
                                    break
                                elif (Choice == "myself"):
                                    print("\nPlease take your sugar packet...")
                                    break
                                else:
                                    print("Please select one of the options\n")
                                    continue
                            except:
                                print("Please select one of the options\n")
                                continue
                        
                        break
                    
                    elif (Choice == "no"):
                        break
                    
                    elif (tempMenu['Stock'][7] == 0):
                        print("Sorry, sugar is currently unavailable")
                        break
                    else:
                        print("Please select yes or no")
                        continue
                except:
                    print("Please select yes or no")
                    continue
                    
            print(f"\nPlease wait...your {key[2:]}(s) will be ready in {Order[key]*10} seconds")

## Change Dispenser
This function serves to dispense change based on available coins in the till

In [12]:
def CalculateChange(AmountPaid, AmountDue, CoinsInserted):
    Coins = [200, 100, 50, 20, 10] # cent value of the coins
    CoinNames = ['$2', '$1', '50c', '20c', '10c'] 
    CoinQuantities = Till['Quantity'].tolist()
    Amount = math.ceil((AmountPaid - AmountDue) * 100)
    Change = {}
    
    for i in range(len(Coins)):
        CoinQuantities[i] = CoinQuantities[i] + CoinsInserted[i]
    
    if (AmountPaid > AmountDue):
        for i in range(len(Coins)):
            CoinCount = min(int(Amount / Coins[i]), CoinQuantities[i])
            if (CoinCount > 0):
                Change[CoinNames[i]] = CoinCount
                Amount -= CoinCount * Coins[i]
                CoinQuantities[i] = CoinQuantities[i] - CoinCount

        if (Amount > 0):
            ChangeFailed = True 
            print("Sorry there is insufficient cash in the till to " 
                  "dispense change, cancelling transaction please take your money.")
            return "Failed"

        elif (Amount == 0):
            print("Change:")
            for key, value in Change.items():
                print(f"{value}x {key} coin(s)")
                
            Till['Quantity'] = CoinQuantities
            Till.to_csv('Till.csv', index=False)

## Transaction Recorder
This function records successful transactions and saves them to the system.
It takes 3 inputs, int AmountPaid, Order and a list of CoinsPaid.

In [13]:
def LogTransaction(AmountPaid, CoinsPaid, Order):
    Date = datetime.now().date()
    FormattedDate = Date.strftime('%d/%m')
    Time = datetime.now().time()
    FormattedTime = Time.strftime('%H:%M')
    PaymentCoins = {}
    TotalPayment = f"${AmountPaid}"
    Items = {}
    
    for i in range(0,5):
        Qty = CoinsPaid[i]
        if (i == 0 and CoinsPaid[i] > 0):
            PaymentCoins[f"{Qty}x"] = "$2 coins"
        elif (i == 1 and CoinsPaid[i] > 0):
            PaymentCoins[f"{Qty}x"] = "$1 coins"
        elif (i == 2 and CoinsPaid[i] > 0):
            PaymentCoins[f"{Qty}x"] = "50c coins"
        elif (i == 3 and CoinsPaid[i] > 0):
            PaymentCoins[f"{Qty}x"] = "20c coins"
        elif (i == 4 and CoinsPaid[i] > 0):
            PaymentCoins[f"{Qty}x"] = "10c coins"
    
    for key,value in Order.items():
        key = key[2:]
        value = f"x{value}"
        Items[key] = value
    
    
    data = {"Date": FormattedDate, "Time": FormattedTime, "Items":Items, "Total":TotalPayment, "Coins":PaymentCoins}
    df = pd.read_csv("Records.csv")
    df = df.append(data, ignore_index=True)
    df.to_csv('Records.csv', index=False)

### New Transaction
Display the welcome message and begin a new transaction

In [49]:
def NewTransaction(menu):
    Transaction = True
    while(Transaction):
        # Welcome messages
        print("----------------------------------")
        print("Welcome!")
        print("\nWhat would you like to buy?\n")

        # Transaction variables
        AmountPaid = 0
        CoinsPaid = [0,0,0,0,0]
        tempMenu = copy.deepcopy(menu)
        PaymentComplete = False

        # Display Items on menu and prompt item selection
        DisplayMenu(tempMenu)
        Order = ItemSelection(tempMenu)
        Button = Options("To Select Items")
        if(Button == "r"):
            continue
        elif(Button == "c"):
            Transaction = False
            return menu

        # Display the cost of each selection and transaction total
        TotalCost = DisplayCost(Order)
        Button = Options("To Payment Screen")
        if(Button == "r"):
            continue
        elif(Button == "c"):
            Transaction = False
            break


        # Payment screen, accept coins and validate payment
        Payment = ""
        while not (PaymentComplete):
            Payment = CoinSlot(TotalCost,CoinsPaid)
            if(Payment == "r"):
                PaymentComplete = True
                break
            elif(Payment == "c"):
                Transaction = False
                break
            elif(Payment == "f"):
                Transaction = False
                break
            elif(Payment == "m"):
                ModifyCart(Order)
                
                # If all items are removed in cart modifications cancel
                if(len(Order) == 0):
                    print("Transaction cancelled.")
                    Transaction = False
                    break
                else:
                    TotalCost = DisplayCost()
            else:
                AmountPaid = Payment
                PaymentComplete = True
        
        if(Payment == "r"):
            continue

        # If payment successful, payment was more than or equal to cost
        elif(AmountPaid >= TotalCost):

            # Sugar Options
            SugarOptions(Order, tempMenu)
            Button = Options("With The Purchase")
            if(Button == "r"):
                continue
            elif(Button == "c"):
                Transaction = False
                print("\nPurchase cancelled. Please take your change")
                break
        
            # Goodbye message
            print("Transaction complete! Goodbye.")
            LogTransaction(AmountPaid,CoinsPaid,Order)
        
            # Update menu    
            return tempMenu

## Admin
This function serves to provide admin controls for the vending machine. Admin can set machine status, view finacial records etc.

In [35]:
def Admin():
    print("\n----Admin Menu----")
    print("\nA. Activate/Deactivate maintenance mode"
          "\nB. Reset system (default amount of change and stock)"
          "\nC. View financial records/history"
          "\nD. View Sales Diagram"
          "\nE. Exit.")
    
    while True:
        try:
            Choice = input("Enter: ").lower()
            
            if (Choice == "a"):
                return "a"
            elif (Choice == "b"):
                return "b"
            elif (Choice == "c"):
                Data = ReadRecords()
                print(tabulate(Data, headers='keys', tablefmt='psql'))
                continue
            elif (Choice == "d"):
                Diagram()
                continue
            elif (Choice == "e"):
                return "e"
            else:
                print("Please select one of the options")
                continue
        except:
            print("Please select one of the options")
            continue

## Menu

In [19]:
Maintenance = False
ChangeFailed = False

while True:
    Choice = ''
    
    if ((Till["Quantity"].astype(str) == "0").any() or ChangeFailed == True):
        Maintenance = True
        print("MAINTENANCE ALERT: The till needs refilling..")
    
    try:
        Choice = input("Type:\n"
                       "- Enter to start transaction\n" 
                       "- Admin passcode to view admin menu\n"
                       "- E to exit\n").lower()
    except:
        print("Please type enter to start")
        continue
        
    if (Choice == "enter" and Maintenance == True):
        print("System is currently under maintenance, sorry for the inconvenience")
        continue
    
    elif (Choice == "enter" and Maintenance == False):
        menu = NewTransaction(menu)
        df = pd.DataFrame(menu)
        df.to_csv('Stock.csv', index=False)
        
    elif (Choice == "admin"):
        AdminSelection = Admin()
        if (AdminSelection == "a"):
            Maintenance = not Maintenance 
            continue
        elif (AdminSelection == "b"):
            ResetStock()
            ResetTill()
            Stock = ReadStock()
            Till = ReadTill()
            menu = CreateMenu()
            Maintenance = False
            continue
        elif (AdminSelection == "e"):
            continue
        
    elif (Choice == "e"):
        break
        
    else:
        print("Please type enter")
        continue
        

Type:
- Enter to start transaction
- Admin passcode to view admin menu
- E to exit
e
